In [2]:
import os
import time
import warnings
from datetime import datetime, timedelta
import pandas as pd
import requests
import yfinance as yf
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from binance.client import Client

warnings.filterwarnings('ignore')

START_DATE = "2017-01-01"
OUTPUT_DIR = "./macro_data_4h"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# [원칙] 실행 시점보다 미래인 데이터는 무조건 삭제
NOW = pd.Timestamp.now()

UPBIT_TICKERS = {
    'KRW-BTC': ('BTC', 'BTC'), 'KRW-ETH': ('ETH', 'ETH'), 'KRW-XRP': ('XRP', 'XRP'),
    'KRW-SOL': ('SOL', 'SOL'), 'KRW-ADA': ('ADA', 'ADA'), 'KRW-DOGE': ('DOGE', 'DOGE'),
    'KRW-AVAX': ('AVAX', 'AVAX'), 'KRW-DOT': ('DOT', 'DOT')
}

BINANCE_SYMBOLS = {
    'BTCUSDT': 'BTC', 'ETHUSDT': 'ETH', 'XRPUSDT': 'XRP', 'SOLUSDT': 'SOL',
    'ADAUSDT': 'ADA', 'DOGEUSDT': 'DOGE', 'AVAXUSDT': 'AVAX', 'DOTUSDT': 'DOT'
}

MACRO_TICKERS = {
    'DX-Y.NYB': 'DXY', 'GC=F': 'GOLD', '^VIX': 'VIX', '^GSPC': 'SP500'
}

DEFI_PROTOCOLS = ['makerdao', 'lido', 'aave', 'uniswap', 'curve-dex']
L2_CHAINS = ['Arbitrum', 'Optimism', 'Base', 'zkSync Era']

def get_session():
    session = requests.Session()
    session.headers.update({
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json"
    })
    retry = Retry(total=5, backoff_factor=2, status_forcelist=[429, 500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retry))
    return session

# -----------------------------------------------------------------------------
# [1] Upbit (기준 데이터)
# -----------------------------------------------------------------------------
def collect_upbit_crypto_prices_4h():
    print(f"\n[1/7] Collecting Upbit (Safe Cut: <= NOW)...")
    session = get_session()
    start_dt = pd.to_datetime(START_DATE)
    merged_df = None
    
    for market, (symbol, _) in UPBIT_TICKERS.items():
        try:
            all_candles = []
            to_date = None
            while True:
                url = "https://api.upbit.com/v1/candles/minutes/240"
                params = {'market': market, 'count': 200}
                if to_date: params['to'] = to_date
                resp = session.get(url, params=params, timeout=10)
                candles = resp.json()
                if not candles: break
                all_candles.extend(candles)
                if pd.to_datetime(candles[-1]['candle_date_time_kst']) <= start_dt: break
                to_date = candles[-1]['candle_date_time_utc']
                time.sleep(0.1)
            
            if not all_candles: continue
            
            df = pd.DataFrame(all_candles)
            df['timestamp'] = pd.to_datetime(df['candle_date_time_kst'])
            
            # [원칙 적용] 미래 데이터 삭제
            df = df[df['timestamp'] <= NOW]
            
            df = df.rename(columns={
                'opening_price': f'{symbol}_Open', 'high_price': f'{symbol}_High',
                'low_price': f'{symbol}_Low', 'trade_price': f'{symbol}_Close',
                'candle_acc_trade_volume': f'{symbol}_Volume'
            })
            df = df[['timestamp', f'{symbol}_Open', f'{symbol}_High', f'{symbol}_Low', f'{symbol}_Close', f'{symbol}_Volume']]
            df = df.sort_values('timestamp').drop_duplicates('timestamp')
            df = df[df['timestamp'] >= start_dt]
            
            merged_df = df if merged_df is None else pd.merge(merged_df, df, on='timestamp', how='outer')
            print(f"  - {symbol}: {len(df)} rows")
        except Exception as e: print(f"  - {symbol}: Failed {e}")
    
    if merged_df is not None:
        merged_df = merged_df.sort_values('timestamp')
        merged_df.to_csv(os.path.join(OUTPUT_DIR, "crypto_4h_kst.csv"), index=False)

# -----------------------------------------------------------------------------
# [2] Binance (KST Aligned & Safe Cut)
# -----------------------------------------------------------------------------
def collect_binance_crypto_prices_4h():
    print(f"\n[2/7] Collecting Binance (KST Aligned & Safe Cut)...")
    client = Client("", "")
    start_ms = int(datetime.strptime(START_DATE, "%Y-%m-%d").timestamp() * 1000)
    merged_df = None

    for symbol, base in BINANCE_SYMBOLS.items():
        try:
            klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_4HOUR, start_ms)
            df = pd.DataFrame(klines, columns=["open_time", "o", "h", "l", "c", "v", "ct", "qav", "nt", "tbb", "tbq", "ig"])
            
            # UTC -> KST (+9h)
            df["timestamp"] = pd.to_datetime(df["open_time"], unit="ms") + pd.Timedelta(hours=9)
            
            # [원칙 적용] 미래 데이터 삭제
            df = df[df['timestamp'] <= NOW]
            
            # 업비트 시간(1, 5, 9...) 정렬
            df = df[df['timestamp'].dt.hour % 4 == 1]
            
            df = df.rename(columns={
                "o": f"{base}_Bin_Open", "h": f"{base}_Bin_High",
                "l": f"{base}_Bin_Low", "c": f"{base}_Bin_Close",
                "v": f"{base}_Bin_Volume"
            })
            df = df[['timestamp', f'{base}_Bin_Open', f'{base}_Bin_High', f'{base}_Bin_Low', f'{base}_Bin_Close', f'{base}_Bin_Vol']]
            df = df[df["timestamp"] >= pd.to_datetime(START_DATE)]
            
            merged_df = df if merged_df is None else pd.merge(merged_df, df, on="timestamp", how="outer")
            print(f"  - {symbol}: {len(df)} rows")
        except: pass
            
    if merged_df is not None:
        merged_df = merged_df.sort_values("timestamp")
        merged_df.to_csv(os.path.join(OUTPUT_DIR, "crypto_binance_4h_kst.csv"), index=False)

# -----------------------------------------------------------------------------
# [3] Macro (Shifted & Safe Cut)
# -----------------------------------------------------------------------------
def collect_macro_indicators_4h():
    print(f"\n[3/7] Collecting Macro (Shifted & Safe Cut)...")
    for ticker, name in MACRO_TICKERS.items():
        try:
            df = yf.download(ticker, start=START_DATE, end=None, progress=False, interval='1d')
            if isinstance(df.columns, pd.MultiIndex): df = df.xs('Close', level=0, axis=1)
            else: df = df[['Close']]
            
            df.index = pd.to_datetime(df.index).tz_localize(None)
            df.columns = [name]
            df.index.name = 'timestamp'

            # 1일 + 9시간 Shift
            df.index = df.index + pd.Timedelta(days=1, hours=9)
            
            # [원칙 적용] Shift 후 미래로 넘어간 데이터(내일 아침 등) 삭제
            # 오늘 장중 데이터 -> 내일 아침으로 밀림 -> 여기서 잘림 -> 안전
            df = df[df.index <= NOW]
            
            # ffill로 현재 시점까지 채움
            full_idx = pd.date_range(start=df.index[0], end=NOW, freq='1H')
            df = df.reindex(full_idx, method='ffill')
            
            # 업비트 시간 추출
            df_4h = df[df.index.hour % 4 == 1].copy()
            
            df_4h = df_4h[df_4h.index >= pd.to_datetime(START_DATE)]
            df_4h.to_csv(os.path.join(OUTPUT_DIR, f"{name}_4h.csv"))
            print(f"  - {name}: {len(df_4h)} rows")
        except Exception as e:
            print(f"  - {name}: Failed {e}")

# -----------------------------------------------------------------------------
# [4] Fear & Greed (Shifted & Safe Cut)
# -----------------------------------------------------------------------------
def collect_fear_greed_4h():
    print(f"\n[4/7] Collecting Fear & Greed (Shifted & Safe Cut)...")
    try:
        session = get_session()
        resp = session.get("https://api.alternative.me/fng/?limit=4000&format=json", timeout=10)
        data = resp.json()['data']
        
        df = pd.DataFrame(data)
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
        df = df[['timestamp', 'value']].rename(columns={'value': 'fear_greed'})
        df['fear_greed'] = df['fear_greed'].astype(float)
        df = df.set_index('timestamp').sort_index()
        
        df.index = df.index + pd.Timedelta(days=1, hours=9)
        
        # [원칙 적용] 미래 데이터 삭제
        df = df[df.index <= NOW]
        
        full_idx = pd.date_range(start=df.index[0], end=NOW, freq='1H')
        df = df.reindex(full_idx, method='ffill')
        
        df_4h = df[df.index.hour % 4 == 1].copy()
        df_4h = df_4h[df_4h.index >= pd.to_datetime(START_DATE)].reset_index()
        
        df_4h.to_csv(os.path.join(OUTPUT_DIR, "fear_greed_4h.csv"), index=False)
        print(f"  - Fear & Greed: {len(df_4h)} rows")
    except Exception as e:
        print(f"  - Fear & Greed: Failed {e}")

# -----------------------------------------------------------------------------
# [5] Funding Rate (KST Aligned & Safe Cut)
# -----------------------------------------------------------------------------
def collect_funding_rate_4h():
    print(f"\n[5/7] Collecting ETH Funding Rate (KST Aligned & Safe Cut)...")
    try:
        client = Client("", "")
        funding_rates = []
        start_ts = int(datetime.strptime(START_DATE, "%Y-%m-%d").timestamp() * 1000)
        end_ts = int(datetime.now().timestamp() * 1000) # 현재까지만
        
        while start_ts < end_ts:
            rates = client.futures_funding_rate(symbol='ETHUSDT', startTime=start_ts, limit=1000)
            if not rates: break
            funding_rates.extend(rates)
            start_ts = rates[-1]['fundingTime'] + 1
            time.sleep(0.1)
        
        df = pd.DataFrame(funding_rates)
        df['timestamp'] = pd.to_datetime(df['fundingTime'], unit='ms') + pd.Timedelta(hours=9)
        df['fundingRate'] = df['fundingRate'].astype(float)
        df = df[['timestamp', 'fundingRate']].sort_values('timestamp').set_index('timestamp')
        
        # [원칙 적용] 미래 데이터 삭제
        df = df[df.index <= NOW]
        
        full_idx = pd.date_range(start=df.index[0], end=NOW, freq='1H')
        df = df.reindex(full_idx, method='ffill')
        
        df_4h = df[df.index.hour % 4 == 1].copy()
        df_4h = df_4h[df_4h.index >= pd.to_datetime(START_DATE)].reset_index()
        
        df_4h.to_csv(os.path.join(OUTPUT_DIR, "eth_funding_rate_4h.csv"), index=False)
        print(f"  - Funding Rate: {len(df_4h)} rows")
    except Exception as e:
        print(f"  - Funding Rate: Failed {e}")

# -----------------------------------------------------------------------------
# [6, 7] TVL (Shifted & Safe Cut)
# -----------------------------------------------------------------------------
def collect_tvl_all():
    print(f"\n[6~7/7] Collecting TVL (Shifted & Safe Cut)...")
    
    def get_data_manual(url):
        session = get_session()
        for i in range(3):
            try:
                time.sleep(2) 
                resp = session.get(url, timeout=30)
                if resp.status_code == 200: return resp.json()
                time.sleep(5)
            except: time.sleep(5)
        return None

    def process_and_save(df, filename, col_name):
        if df is None or df.empty: return
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        if df['timestamp'].dt.tz is not None:
            df['timestamp'] = df['timestamp'].dt.tz_localize(None)
            
        # 1일 + 9시간 Shift
        df['timestamp'] = df['timestamp'] + pd.Timedelta(days=1, hours=9)
        
        df = df.sort_values('timestamp').drop_duplicates(subset=['timestamp']).set_index('timestamp')
        
        # [원칙 적용] 미래 데이터 삭제
        df = df[df.index <= NOW]
        
        # ffill로 현재까지 채움
        full_idx = pd.date_range(start=df.index[0], end=NOW, freq='1H')
        df = df.reindex(full_idx, method='ffill')
        
        df_4h = df[df.index.hour % 4 == 1].copy()
        
        df_4h = df_4h[df_4h.index >= pd.to_datetime(START_DATE)].reset_index()
        df_4h.to_csv(os.path.join(OUTPUT_DIR, filename), index=False)
        print(f"  - {col_name}: {len(df_4h)} rows")

    # ETH Chain
    try:
        data = get_data_manual("https://api.llama.fi/v2/historicalChainTvl/Ethereum")
        if data:
            df = pd.DataFrame(data)
            df['timestamp'] = pd.to_datetime(df['date'], unit='s')
            df = df.rename(columns={'tvl': 'eth_chain_tvl'})[['timestamp', 'eth_chain_tvl']]
            process_and_save(df, 'eth_chain_tvl_4h.csv', 'ETH Chain TVL')
    except: pass

    # Protocols
    for protocol in DEFI_PROTOCOLS:
        try:
            data = get_data_manual(f"https://api.llama.fi/protocol/{protocol}")
            if data:
                chain_data = data.get('chainTvls', {}).get('Ethereum', {}).get('tvl', [])
                if not chain_data: chain_data = data.get('tvl', [])
                if chain_data:
                    df = pd.DataFrame(chain_data)
                    df['timestamp'] = pd.to_datetime(df['date'], unit='s')
                    val_col = 'totalLiquidityUSD' if 'totalLiquidityUSD' in df.columns else 'tvl'
                    df = df.rename(columns={val_col: f'{protocol}_eth_tvl'})
                    process_and_save(df[['timestamp', f'{protocol}_eth_tvl']], f'{protocol}_eth_tvl_4h.csv', protocol)
        except: pass

    # L2 Chains
    for chain in L2_CHAINS:
        try:
            chain_name = chain.replace(' ', '_').lower()
            data = get_data_manual(f"https://api.llama.fi/v2/historicalChainTvl/{chain}")
            if data:
                df = pd.DataFrame(data)
                df['timestamp'] = pd.to_datetime(df['date'], unit='s')
                df = df.rename(columns={'tvl': f'{chain_name}_tvl'})
                process_and_save(df[['timestamp', f'{chain_name}_tvl']], f'{chain_name}_tvl_4h.csv', chain)
        except: pass
    
    # USDT Mcap
    try:
        data = get_data_manual("https://stablecoins.llama.fi/stablecoincharts/Ethereum?stablecoin=1")
        if data:
            df = pd.DataFrame(data)

            if 'totalCirculatingUSD' in df.columns:
                df['usdt_eth_mcap'] = df['totalCirculatingUSD'].apply(
                    lambda x: x.get('peggedUSD') if isinstance(x, dict) else x
                )
                
                # 2. 타임스탬프 처리
                df['timestamp'] = pd.to_datetime(df['date'], unit='s')
                
                # 3. 저장
                process_and_save(df[['timestamp', 'usdt_eth_mcap']], 'usdt_eth_mcap_4h.csv', 'USDT Mcap')
    except Exception as e:
        print(f"  - USDT Mcap: Failed ({str(e)[:50]})")

# -----------------------------------------------------------------------------
# Run All
# -----------------------------------------------------------------------------
print("🚀 Starting Final Pipeline (Safe Cut Rule Applied)...")
collect_upbit_crypto_prices_4h()
collect_binance_crypto_prices_4h()
collect_macro_indicators_4h()
collect_fear_greed_4h()
collect_funding_rate_4h()
collect_tvl_all()
print("✅ Done.")



🚀 Starting Final Pipeline (Safe Cut Rule Applied)...

[1/7] Collecting Upbit (Safe Cut: <= NOW)...
  - BTC: 17936 rows
  - ETH: 17913 rows
  - XRP: 17912 rows
  - SOL: 9059 rows
  - ADA: 17852 rows
  - DOGE: 10457 rows
  - AVAX: 8362 rows
  - DOT: 11309 rows

[2/7] Collecting Binance (KST Aligned & Safe Cut)...

[3/7] Collecting Macro (Shifted & Safe Cut)...
  - DXY: 19531 rows
  - GOLD: 19531 rows
  - VIX: 19531 rows
  - SP500: 19531 rows

[4/7] Collecting Fear & Greed (Shifted & Safe Cut)...
  - Fear & Greed: 17167 rows

[5/7] Collecting ETH Funding Rate (KST Aligned & Safe Cut)...
  - Funding Rate: 13187 rows

[6~7/7] Collecting TVL (Shifted & Safe Cut)...
  - ETH Chain TVL: 17929 rows
  - makerdao: 15145 rows
  - lido: 10849 rows
  - aave: 12133 rows
  - curve-dex: 12739 rows
  - Arbitrum: 9853 rows
  - Optimism: 9613 rows
  - Base: 5407 rows
  - zkSync Era: 7801 rows
  - USDT Mcap: 17551 rows
✅ Done.


In [17]:
def get_data_manual(url):
    session = get_session()
    for i in range(3):
        try:
            time.sleep(2) 
            resp = session.get(url, timeout=30)
            if resp.status_code == 200: return resp.json()
            time.sleep(5)
        except: time.sleep(5)
    return None

def process_and_save(df, filename, col_name):
    if df is None or df.empty: return
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    if df['timestamp'].dt.tz is not None:
        df['timestamp'] = df['timestamp'].dt.tz_localize(None)

    # 1일 + 9시간 Shift
    df['timestamp'] = df['timestamp'] + pd.Timedelta(days=1, hours=9)

    df = df.sort_values('timestamp').drop_duplicates(subset=['timestamp']).set_index('timestamp')

    # [원칙 적용] 미래 데이터 삭제
    df = df[df.index <= NOW]

    # ffill로 현재까지 채움
    full_idx = pd.date_range(start=df.index[0], end=NOW, freq='1H')
    df = df.reindex(full_idx, method='ffill')

    df_4h = df[df.index.hour % 4 == 1].copy()

    df_4h = df_4h[df_4h.index >= pd.to_datetime(START_DATE)].reset_index()
    df_4h.to_csv(os.path.join(OUTPUT_DIR, filename), index=False)
    print(f"  - {col_name}: {len(df_4h)} rows")

# ETH Chain
try:
    data = get_data_manual("https://api.llama.fi/v2/historicalChainTvl/Ethereum")
    if data:
        df = pd.DataFrame(data)
        df['timestamp'] = pd.to_datetime(df['date'], unit='s')
        df = df.rename(columns={'tvl': 'eth_chain_tvl'})[['timestamp', 'eth_chain_tvl']]
        process_and_save(df, 'eth_chain_tvl_4h.csv', 'ETH Chain TVL')
except: pass
    
data = get_data_manual("https://stablecoins.llama.fi/stablecoincharts/Ethereum?stablecoin=1")

df = pd.DataFrame(data)

if 'totalCirculatingUSD' in df.columns:
    df['usdt_eth_mcap'] = df['totalCirculatingUSD'].apply(
        lambda x: x.get('peggedUSD') if isinstance(x, dict) else x)
    # 2. 타임스탬프 처리
    df['timestamp'] = pd.to_datetime(df['date'], unit='s')

    # 3. 저장
    process_and_save(df[['timestamp', 'usdt_eth_mcap']], 'usdt_eth_mcap_4h.csv', 'USDT Mcap')

  - ETH Chain TVL: 17902 rows
  - USDT Mcap: 17524 rows


In [1]:
import os
import requests
import pandas as pd


# ==============================
# 설정
# ==============================
OUTPUT_DIR = "./macro_data_4h"
FINAL_FILENAME = "eth_4hour.csv"

# FRED API
FRED_API_KEY = "eab5d975cd0091d7ab3f56cdda8e6d22"
FRED_BASE_URL = "https://api.stlouisfed.org/fred/series/observations"


series_map = {
    "M2SL": "fred_M2SL",          # M2 Money Stock
    "WALCL": "fred_WALCL",        # Fed Balance Sheet Total Assets
    "RRPONTSYD": "fred_RRPONTSYD",# Overnight Reverse Repo
    "EFFR": "fred_EFFR",          # Effective Fed Funds Rate
    "DGS10": "fred_DGS10",        # 10Y Treasury Yield
    "DTWEXBGS": "fred_DTWEXBGS",  # Broad Dollar Index
    "CPIAUCSL": "fred_CPI",       # CPI: All items, urban consumers
}


def fetch_fred_series(series_id, start_date="2017-01-01"):
    params = {
        "series_id": series_id,
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "observation_start": start_date,
        "sort_order": "asc",
    }
    r = requests.get(FRED_BASE_URL, params=params)
    print("DEBUG status", series_id, r.status_code)
    r.raise_for_status()

    data = r.json()["observations"]
    df = pd.DataFrame(data)
    print("DEBUG raw head", series_id)
    print(df.head(5)[["date", "value"]])

    df["date"] = pd.to_datetime(df["date"])
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    print("DEBUG numeric head", series_id)
    print(df.head(5)[["date", "value"]])

    df = df[["date", "value"]].dropna()
    print("DEBUG n_rows after dropna", series_id, len(df))
    return df


def build_fred_macro_on_base(base_timestamps, start_date="2017-01-01"):
    """
    FRED 일일 데이터를 4H 타임라인에 맞추고, 
    단순 ffill 대신 '유의미한 파생 변수(Diff, Pct, Trend)'를 생성하여 병합함.
    """
    idx = pd.to_datetime(base_timestamps).sort_values().unique()
    
    # 최종 결과를 담을 리스트 (원본 값은 나중에 지우거나 참고용으로만 씀)
    final_features = []

    for series_id, col_name in series_map.items():
        print(f"   -> Fetching & Engineering FRED series: {series_id} ({col_name})")
        

        df_daily = fetch_fred_series(series_id, start_date=start_date)
        df = df_daily.copy()
        
        # 2. 날짜 시프트 (미국 d일 -> KST d+1일 09:00 적용)
        # 이 시점에서는 아직 '일봉(Daily)' 상태입니다.
        df["ts_eff_kst"] = df["date"] + pd.Timedelta(days=1, hours=9)
        df.set_index("ts_eff_kst", inplace=True)
        df.sort_index(inplace=True)
        
        # ==================================================================
        # [핵심] 일봉 상태에서 미리 지표를 계산해야 노이즈가 없습니다.
        # 4시간봉으로 늘린 뒤에 계산하면 0이 5개 나오고 1개만 튀는 문제 발생함.
        # ==================================================================
        
        # (1) 전일 대비 변화량 (Diff) - 금리, 지수 같은 포인트 단위
        # 계단식 문제 해결: 0이 아니라 '어제와 오늘의 차이'가 4시간 동안 유지됨 -> "금리 상승 기조"로 인식
        s_diff = df['value'].diff() 
        
        # (2) 전일 대비 변화율 (Pct Change) - M2, 대차대조표 같은 큰 숫자 단위
        s_pct = df['value'].pct_change()
        
        # (3) 추세 이격도 (Trend)
        s_ma20 = df['value'].rolling(window=20).mean()
        s_trend_20 = (df['value'] - s_ma20) / (s_ma20 + 1e-9)

        # [추가 추천] 60일(분기) 장기 추세 추가
        s_ma60 = df['value'].rolling(window=60).mean()
        s_trend_60 = (df['value'] - s_ma60) / (s_ma60 + 1e-9)
        
        features_to_map = {
            # ... (기존과 동일)
            f"{col_name}_diff_1d": s_diff,
            f"{col_name}_pct_1d": s_pct,
            f"{col_name}_trend_20d": s_trend_20,
            f"{col_name}_trend_60d": s_trend_60, # <--- 이거 하나 추가!
        }

        for feat_name, s_daily in features_to_map.items():

            s_on_base = s_daily.reindex(idx, method="ffill")
            s_on_base.name = feat_name
            final_features.append(s_on_base)

    # 4. 데이터프레임 통합
    df_macro = pd.concat(final_features, axis=1)
    df_macro = df_macro.reset_index().rename(columns={"index": "timestamp"})
    
    # NaN 처리 (앞부분 이동평균 계산 기간 등)
    # 0으로 채우면 '변화 없음'으로 인식되어 안전함
    df_macro = df_macro.fillna(0)
    
    return df_macro


def build_and_save_fred_only(base_path, start_date="2017-01-01"):
    if not os.path.exists(base_path):
        print(f"Error: Base file not found: {base_path}")
        return

    base_df = pd.read_csv(base_path)
    if "timestamp" not in base_df.columns:
        base_df.rename(columns={base_df.columns[0]: "timestamp"}, inplace=True)
    base_df["timestamp"] = pd.to_datetime(base_df["timestamp"])
    base_df = base_df.sort_values("timestamp")

    print(f"Base Timeline: {os.path.basename(base_path)}")
    print(f"   -> Start Date: {base_df['timestamp'].min()}")
    print(f"   -> End Date:   {base_df['timestamp'].max()}")
    print(f"   -> Total Rows: {len(base_df)}")

    print("Building FRED macro features aligned to base timeline...")
    df_fred_on_base = build_fred_macro_on_base(
        base_df["timestamp"],
        start_date=start_date,
    )

    print("DEBUG FRED on base head")
    print(df_fred_on_base.head())

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    save_path = os.path.join(OUTPUT_DIR, "fred_macro_4h.csv")
    df_fred_on_base.to_csv(save_path, index=False)
    print(f"Saved FRED macro file: {os.path.basename(save_path)}")


# 실행
base_path = os.path.join(OUTPUT_DIR, "crypto_4h_kst.csv")
build_and_save_fred_only(base_path, start_date="2017-01-01")

Base Timeline: crypto_4h_kst.csv
   -> Start Date: 2017-09-26 17:00:00
   -> End Date:   2025-12-01 13:00:00
   -> Total Rows: 17925
Building FRED macro features aligned to base timeline...
   -> Fetching & Engineering FRED series: M2SL (fred_M2SL)
DEBUG status M2SL 200
DEBUG raw head M2SL
         date    value
0  2017-01-01  13300.3
1  2017-02-01  13367.0
2  2017-03-01  13428.4
3  2017-04-01  13484.4
4  2017-05-01  13536.4
DEBUG numeric head M2SL
        date    value
0 2017-01-01  13300.3
1 2017-02-01  13367.0
2 2017-03-01  13428.4
3 2017-04-01  13484.4
4 2017-05-01  13536.4
DEBUG n_rows after dropna M2SL 106
   -> Fetching & Engineering FRED series: WALCL (fred_WALCL)
DEBUG status WALCL 200
DEBUG raw head WALCL
         date    value
0  2017-01-04  4453101
1  2017-01-11  4452509
2  2017-01-18  4460758
3  2017-01-25  4452838
4  2017-02-01  4453881
DEBUG numeric head WALCL
        date      value
0 2017-01-04  4453101.0
1 2017-01-11  4452509.0
2 2017-01-18  4460758.0
3 2017-01-25  44

In [23]:
fred4 = pd.read_csv("./macro_data_4h/fred_macro_4h.csv")
print(fred4.head(10))
print("n_unique CPI:", fred4["fred_CPI"].nunique(),
      "na_ratio:", fred4["fred_CPI"].isna().mean())


            ts_eff_kst  fred_M2SL  fred_WALCL  fred_RRPONTSYD  fred_EFFR  \
0  2017-09-26 17:00:00    13722.7   4458576.0         158.503       1.16   
1  2017-09-26 21:00:00    13722.7   4458576.0         158.503       1.16   
2  2017-09-27 01:00:00    13722.7   4458576.0         158.503       1.16   
3  2017-09-27 05:00:00    13722.7   4458576.0         158.503       1.16   
4  2017-09-27 09:00:00    13722.7   4458576.0         173.715       1.16   
5  2017-09-27 13:00:00    13722.7   4458576.0         173.715       1.16   
6  2017-09-27 17:00:00    13722.7   4458576.0         173.715       1.16   
7  2017-09-27 21:00:00    13722.7   4458576.0         173.715       1.16   
8  2017-09-28 01:00:00    13722.7   4458576.0         173.715       1.16   
9  2017-09-28 05:00:00    13722.7   4458576.0         173.715       1.16   

   fred_DGS10  fred_DTWEXBGS  fred_CPI  
0        2.22       109.3302   246.435  
1        2.22       109.3302   246.435  
2        2.22       109.3302   246.435  

In [3]:
import os
import glob
import pandas as pd

OUTPUT_DIR = "./macro_data_4h"
FINAL_FILENAME = "eth_4hour.csv"


def create_eth_dataset():
    print("🔄 Starting Final Merge (Target: eth_4hour.csv)...")

    base_path = os.path.join(OUTPUT_DIR, "crypto_4h_kst.csv")
    if not os.path.exists(base_path):
        print("❌ Error: Base file 'crypto_4h_kst.csv' not found.")
        return

    print(f"📍 Base Timeline: {os.path.basename(base_path)}")
    base_df = pd.read_csv(base_path)

    if "timestamp" not in base_df.columns:
        base_df.rename(columns={base_df.columns[0]: "timestamp"}, inplace=True)
    base_df["timestamp"] = pd.to_datetime(base_df["timestamp"])
    base_df = base_df.sort_values("timestamp")

    print(f"   -> Start Date: {base_df['timestamp'].min()}")
    print(f"   -> End Date:   {base_df['timestamp'].max()}")
    print(f"   -> Total Rows: {len(base_df)}")

    all_files = glob.glob(os.path.join(OUTPUT_DIR, "*.csv"))
    exclude_keywords = [
        "crypto_4h_kst.csv",
        "eth_4hour.csv",
        "final_",
        "merged",
        "upbit_",
    ]

    files_to_merge = []
    for f in all_files:
        fname = os.path.basename(f)
        if not any(k in fname for k in exclude_keywords):
            files_to_merge.append(f)

    print(f"📂 Files to merge: {len(files_to_merge)}")

    for file_path in sorted(files_to_merge):
        fname = os.path.basename(file_path)
        try:
            df = pd.read_csv(file_path)
            if "timestamp" not in df.columns:
                df.rename(columns={df.columns[0]: "timestamp"}, inplace=True)
            df["timestamp"] = pd.to_datetime(df["timestamp"])
            df = df.drop_duplicates("timestamp")
            base_df = pd.merge(base_df, df, on="timestamp", how="left")
            print(f"  ✅ Merged: {fname}")
        except Exception as e:
            print(f"  ⚠️ Error merging {fname}: {e}")

    print("🛠  Handling Missing Values...")
    base_df = base_df.ffill().fillna(0)

    save_path = os.path.join(OUTPUT_DIR, FINAL_FILENAME)
    base_df.to_csv(save_path, index=False)

    print("\n" + "=" * 50)
    print(f"🎉 COMPLETE! File created: {FINAL_FILENAME}")
    print(f"📊 Final Shape: {base_df.shape}")
    print("=" * 50)


if __name__ == "__main__":
    create_eth_dataset()


🔄 Starting Final Merge (Target: eth_4hour.csv)...
📍 Base Timeline: crypto_4h_kst.csv
   -> Start Date: 2017-09-26 17:00:00
   -> End Date:   2025-12-03 09:00:00
   -> Total Rows: 17936
📂 Files to merge: 19
  ✅ Merged: DXY_4h.csv
  ✅ Merged: GOLD_4h.csv
  ✅ Merged: SP500_4h.csv
  ✅ Merged: VIX_4h.csv
  ✅ Merged: aave_eth_tvl_4h.csv
  ✅ Merged: arbitrum_tvl_4h.csv
  ✅ Merged: base_tvl_4h.csv
  ✅ Merged: crypto_binance_4h_kst.csv
  ✅ Merged: curve-dex_eth_tvl_4h.csv
  ✅ Merged: eth_chain_tvl_4h.csv
  ✅ Merged: eth_funding_rate_4h.csv
  ✅ Merged: fear_greed_4h.csv
  ✅ Merged: fred_macro_4h.csv
  ✅ Merged: lido_eth_tvl_4h.csv
  ✅ Merged: makerdao_eth_tvl_4h.csv
  ✅ Merged: optimism_tvl_4h.csv
  ✅ Merged: uniswap_eth_tvl_4h.csv
  ✅ Merged: usdt_eth_mcap_4h.csv
  ✅ Merged: zksync_era_tvl_4h.csv
🛠  Handling Missing Values...

🎉 COMPLETE! File created: eth_4hour.csv
📊 Final Shape: (17936, 126)


In [1]:
import requests
import pandas as pd
import time

def fetch_upbit_hourly(market='KRW-ETH', start_date='2020-01-01'):
    all_data = []
    to_date = pd.Timestamp.now()
    start_ts = pd.to_datetime(start_date)
    
    print(f"Fetching Upbit {market} 1H data: {start_date} ~ now")
    
    while to_date > start_ts:
        url = "https://api.upbit.com/v1/candles/minutes/60"
        params = {
            'market': market,
            'to': to_date.strftime('%Y-%m-%dT%H:%M:%S'),
            'count': 200
        }
        
        try:
            response = requests.get(url, params=params)
            data = response.json()
            
            if not data or isinstance(data, dict):
                break
            
            all_data.extend(data)
            
            oldest = pd.to_datetime(data[-1]['candle_date_time_kst'])
            to_date = oldest
            
            print(f"  Fetched {len(all_data)} candles... (oldest: {oldest})")
            
            time.sleep(0.15)
            
        except Exception as e:
            print(f"  Error: {e}, retrying...")
            time.sleep(1)
    
    df = pd.DataFrame(all_data)
    df = df.rename(columns={
        'candle_date_time_kst': 'datetime',
        'opening_price': 'open',
        'high_price': 'high',
        'low_price': 'low',
        'trade_price': 'close',
        'candle_acc_trade_volume': 'volume'
    })
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('datetime').reset_index(drop=True)
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume']]
    df = df.drop_duplicates(subset='datetime').reset_index(drop=True)
    
    mask = df['datetime'] >= start_date
    df = df[mask].reset_index(drop=True)
    
    print(f"Done! Total {len(df)} hourly candles")
    print(f"Period: {df['datetime'].min()} ~ {df['datetime'].max()}")
    
    return df

eth_hourly_krw = fetch_upbit_hourly('KRW-ETH', '2020-01-01')
eth_hourly_krw.to_csv('eth_hour.csv', index=False)

Fetching Upbit KRW-ETH 1H data: 2020-01-01 ~ now
  Fetched 200 candles... (oldest: 2025-11-24 11:00:00)
  Fetched 400 candles... (oldest: 2025-11-16 12:00:00)
  Fetched 600 candles... (oldest: 2025-11-08 13:00:00)
  Fetched 800 candles... (oldest: 2025-10-31 14:00:00)
  Fetched 1000 candles... (oldest: 2025-10-23 10:00:00)
  Fetched 1200 candles... (oldest: 2025-10-15 11:00:00)
  Fetched 1400 candles... (oldest: 2025-10-07 12:00:00)
  Fetched 1600 candles... (oldest: 2025-09-29 13:00:00)
  Fetched 1800 candles... (oldest: 2025-09-21 14:00:00)
  Fetched 2000 candles... (oldest: 2025-09-13 15:00:00)
  Fetched 2200 candles... (oldest: 2025-09-05 16:00:00)
  Fetched 2400 candles... (oldest: 2025-08-28 17:00:00)
  Fetched 2600 candles... (oldest: 2025-08-20 18:00:00)
  Fetched 2800 candles... (oldest: 2025-08-12 19:00:00)
  Fetched 3000 candles... (oldest: 2025-08-04 20:00:00)
  Fetched 3200 candles... (oldest: 2025-07-27 21:00:00)
  Fetched 3400 candles... (oldest: 2025-07-19 22:00:00)
  F

  Fetched 29000 candles... (oldest: 2022-10-03 06:00:00)
  Fetched 29200 candles... (oldest: 2022-09-25 06:00:00)
  Fetched 29400 candles... (oldest: 2022-09-17 07:00:00)
  Fetched 29600 candles... (oldest: 2022-09-09 08:00:00)
  Fetched 29800 candles... (oldest: 2022-09-01 09:00:00)
  Fetched 30000 candles... (oldest: 2022-08-24 10:00:00)
  Fetched 30200 candles... (oldest: 2022-08-16 11:00:00)
  Fetched 30400 candles... (oldest: 2022-08-08 12:00:00)
  Fetched 30600 candles... (oldest: 2022-07-31 13:00:00)
  Fetched 30800 candles... (oldest: 2022-07-23 14:00:00)
  Fetched 31000 candles... (oldest: 2022-07-15 15:00:00)
  Fetched 31200 candles... (oldest: 2022-07-07 16:00:00)
  Fetched 31400 candles... (oldest: 2022-06-29 14:00:00)
  Fetched 31600 candles... (oldest: 2022-06-21 15:00:00)
  Fetched 31800 candles... (oldest: 2022-06-13 16:00:00)
  Fetched 32000 candles... (oldest: 2022-06-05 17:00:00)
  Fetched 32200 candles... (oldest: 2022-05-28 18:00:00)
  Fetched 32400 candles... (old

In [3]:
import requests
import pandas as pd
import time

def fetch_upbit_5min(market='KRW-ETH', start_date='2020-01-01'):
    all_data = []
    to_date = pd.Timestamp.now()
    start_ts = pd.to_datetime(start_date)
    
    print(f"Fetching Upbit {market} 5min data: {start_date} ~ now")
    
    while to_date > start_ts:
        url = "https://api.upbit.com/v1/candles/minutes/5" 
        params = {
            'market': market,
            'to': to_date.strftime('%Y-%m-%dT%H:%M:%S'),
            'count': 200
        }
        
        try:
            response = requests.get(url, params=params)
            data = response.json()
            
            if not data or isinstance(data, dict):
                break
            
            all_data.extend(data)
            
            oldest = pd.to_datetime(data[-1]['candle_date_time_kst'])
            to_date = oldest
            
            print(f"  Fetched {len(all_data)} candles... (oldest: {oldest})")
            
            time.sleep(0.11)  # 업비트 Rate Limit: 초당 10회 = 0.1초 간격
            
        except Exception as e:
            print(f"  Error: {e}, retrying...")
            time.sleep(1)
    
    df = pd.DataFrame(all_data)
    df = df.rename(columns={
        'candle_date_time_kst': 'date',
        'opening_price': 'ETH_Open',
        'high_price': 'ETH_High',
        'low_price': 'ETH_Low',
        'trade_price': 'ETH_Close',
        'candle_acc_trade_volume': 'ETH_Volume'
    })
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    df = df[['date', 'ETH_Open', 'ETH_High', 'ETH_Low', 'ETH_Close', 'ETH_Volume']]
    
    # ⚠️ 버그 수정 1: 'datetime' → 'date'
    df = df.drop_duplicates(subset='date').reset_index(drop=True)
    
    # ⚠️ 버그 수정 2: 'datetime' → 'date'
    mask = df['date'] >= start_date
    df = df[mask].reset_index(drop=True)
    
    print(f"Done! Total {len(df)} 5-minute candles") 
    print(f"Period: {df['date'].min()} ~ {df['date'].max()}")
    
    return df

# 실행
eth_5min_krw = fetch_upbit_5min('KRW-ETH', '2019-01-01')
eth_5min_krw.to_csv('eth_5min_krw.csv', index=False)




Fetching Upbit KRW-ETH 5min data: 2019-01-01 ~ now
  Fetched 200 candles... (oldest: 2025-12-03 16:50:00)
  Fetched 400 candles... (oldest: 2025-12-03 09:10:00)
  Fetched 600 candles... (oldest: 2025-12-03 01:30:00)
  Fetched 800 candles... (oldest: 2025-12-02 17:50:00)
  Fetched 1000 candles... (oldest: 2025-12-02 10:10:00)
  Fetched 1200 candles... (oldest: 2025-12-02 02:30:00)
  Fetched 1400 candles... (oldest: 2025-12-01 18:50:00)
  Fetched 1600 candles... (oldest: 2025-12-01 11:10:00)
  Fetched 1800 candles... (oldest: 2025-12-01 03:30:00)
  Fetched 2000 candles... (oldest: 2025-11-30 19:50:00)
  Fetched 2200 candles... (oldest: 2025-11-30 12:10:00)
  Fetched 2400 candles... (oldest: 2025-11-30 04:30:00)
  Fetched 2600 candles... (oldest: 2025-11-29 20:50:00)
  Fetched 2800 candles... (oldest: 2025-11-29 13:10:00)
  Fetched 3000 candles... (oldest: 2025-11-29 05:30:00)
  Fetched 3200 candles... (oldest: 2025-11-28 21:50:00)
  Fetched 3400 candles... (oldest: 2025-11-28 14:10:00)
 

  Fetched 29200 candles... (oldest: 2025-10-17 19:30:00)
  Fetched 29400 candles... (oldest: 2025-10-17 11:50:00)
  Fetched 29600 candles... (oldest: 2025-10-17 04:10:00)
  Fetched 29800 candles... (oldest: 2025-10-16 20:30:00)
  Fetched 30000 candles... (oldest: 2025-10-16 12:50:00)
  Fetched 30200 candles... (oldest: 2025-10-16 05:10:00)
  Fetched 30400 candles... (oldest: 2025-10-15 21:30:00)
  Fetched 30600 candles... (oldest: 2025-10-15 13:50:00)
  Fetched 30800 candles... (oldest: 2025-10-15 06:10:00)
  Fetched 31000 candles... (oldest: 2025-10-14 22:30:00)
  Fetched 31200 candles... (oldest: 2025-10-14 14:50:00)
  Fetched 31400 candles... (oldest: 2025-10-14 07:10:00)
  Fetched 31600 candles... (oldest: 2025-10-13 23:30:00)
  Fetched 31800 candles... (oldest: 2025-10-13 15:50:00)
  Fetched 32000 candles... (oldest: 2025-10-13 08:10:00)
  Fetched 32200 candles... (oldest: 2025-10-13 00:30:00)
  Fetched 32400 candles... (oldest: 2025-10-12 16:50:00)
  Fetched 32600 candles... (old

  Fetched 58200 candles... (oldest: 2025-09-01 11:50:00)
  Fetched 58400 candles... (oldest: 2025-09-01 04:10:00)
  Fetched 58600 candles... (oldest: 2025-08-31 20:30:00)
  Fetched 58800 candles... (oldest: 2025-08-31 12:50:00)
  Fetched 59000 candles... (oldest: 2025-08-31 05:10:00)
  Fetched 59200 candles... (oldest: 2025-08-30 21:30:00)
  Fetched 59400 candles... (oldest: 2025-08-30 13:50:00)
  Fetched 59600 candles... (oldest: 2025-08-30 06:10:00)
  Fetched 59800 candles... (oldest: 2025-08-29 22:30:00)
  Fetched 60000 candles... (oldest: 2025-08-29 14:50:00)
  Fetched 60200 candles... (oldest: 2025-08-29 07:10:00)
  Fetched 60400 candles... (oldest: 2025-08-28 23:30:00)
  Fetched 60600 candles... (oldest: 2025-08-28 15:50:00)
  Fetched 60800 candles... (oldest: 2025-08-28 08:10:00)
  Fetched 61000 candles... (oldest: 2025-08-28 00:30:00)
  Fetched 61200 candles... (oldest: 2025-08-27 16:50:00)
  Fetched 61400 candles... (oldest: 2025-08-27 09:10:00)
  Fetched 61600 candles... (old

  Fetched 87000 candles... (oldest: 2025-07-17 11:20:00)
  Fetched 87200 candles... (oldest: 2025-07-17 03:40:00)
  Fetched 87400 candles... (oldest: 2025-07-16 20:00:00)
  Fetched 87600 candles... (oldest: 2025-07-16 12:20:00)
  Fetched 87800 candles... (oldest: 2025-07-16 04:40:00)
  Fetched 88000 candles... (oldest: 2025-07-15 21:00:00)
  Fetched 88200 candles... (oldest: 2025-07-15 13:20:00)
  Fetched 88400 candles... (oldest: 2025-07-15 05:40:00)
  Fetched 88600 candles... (oldest: 2025-07-14 22:00:00)
  Fetched 88800 candles... (oldest: 2025-07-14 14:20:00)
  Fetched 89000 candles... (oldest: 2025-07-14 06:40:00)
  Fetched 89200 candles... (oldest: 2025-07-13 23:00:00)
  Fetched 89400 candles... (oldest: 2025-07-13 15:20:00)
  Fetched 89600 candles... (oldest: 2025-07-13 07:40:00)
  Fetched 89800 candles... (oldest: 2025-07-13 00:00:00)
  Fetched 90000 candles... (oldest: 2025-07-12 16:20:00)
  Fetched 90200 candles... (oldest: 2025-07-12 08:40:00)
  Fetched 90400 candles... (old

  Fetched 115600 candles... (oldest: 2025-06-01 07:45:00)
  Fetched 115800 candles... (oldest: 2025-06-01 00:05:00)
  Fetched 116000 candles... (oldest: 2025-05-31 16:25:00)
  Fetched 116200 candles... (oldest: 2025-05-31 08:45:00)
  Fetched 116400 candles... (oldest: 2025-05-31 01:05:00)
  Fetched 116600 candles... (oldest: 2025-05-30 17:25:00)
  Fetched 116800 candles... (oldest: 2025-05-30 09:45:00)
  Fetched 117000 candles... (oldest: 2025-05-30 02:05:00)
  Fetched 117200 candles... (oldest: 2025-05-29 18:25:00)
  Fetched 117400 candles... (oldest: 2025-05-29 10:45:00)
  Fetched 117600 candles... (oldest: 2025-05-29 03:05:00)
  Fetched 117800 candles... (oldest: 2025-05-28 19:25:00)
  Fetched 118000 candles... (oldest: 2025-05-28 11:45:00)
  Fetched 118200 candles... (oldest: 2025-05-28 04:05:00)
  Fetched 118400 candles... (oldest: 2025-05-27 20:25:00)
  Fetched 118600 candles... (oldest: 2025-05-27 12:45:00)
  Fetched 118800 candles... (oldest: 2025-05-27 05:05:00)
  Fetched 1190

...

  Fetched 1445200 candles... (oldest: 2019-07-22 09:15:00)
  Fetched 1445400 candles... (oldest: 2019-07-22 01:35:00)
  Fetched 1445600 candles... (oldest: 2019-07-21 17:55:00)
  Fetched 1445800 candles... (oldest: 2019-07-21 10:15:00)
  Fetched 1446000 candles... (oldest: 2019-07-21 02:35:00)
  Fetched 1446200 candles... (oldest: 2019-07-20 18:55:00)
  Fetched 1446400 candles... (oldest: 2019-07-20 11:15:00)
  Fetched 1446600 candles... (oldest: 2019-07-20 03:35:00)
  Fetched 1446800 candles... (oldest: 2019-07-19 19:55:00)
  Fetched 1447000 candles... (oldest: 2019-07-19 12:15:00)
  Fetched 1447200 candles... (oldest: 2019-07-19 04:35:00)
  Fetched 1447400 candles... (oldest: 2019-07-18 20:55:00)
  Fetched 1447600 candles... (oldest: 2019-07-18 13:15:00)
  Fetched 1447800 candles... (oldest: 2019-07-18 05:35:00)
  Fetched 1448000 candles... (oldest: 2019-07-17 21:55:00)
  Fetched 1448200 candles... (oldest: 2019-07-17 14:15:00)
  Fetched 1448400 candles... (oldest: 2019-07-17 06:35:0

  Fetched 1473000 candles... (oldest: 2019-06-07 23:00:00)
  Fetched 1473200 candles... (oldest: 2019-06-07 15:20:00)
  Fetched 1473400 candles... (oldest: 2019-06-07 07:40:00)
  Fetched 1473600 candles... (oldest: 2019-06-07 00:00:00)
  Fetched 1473800 candles... (oldest: 2019-06-06 16:20:00)
  Fetched 1474000 candles... (oldest: 2019-06-06 08:40:00)
  Fetched 1474200 candles... (oldest: 2019-06-06 01:00:00)
  Fetched 1474400 candles... (oldest: 2019-06-05 17:20:00)
  Fetched 1474600 candles... (oldest: 2019-06-05 09:40:00)
  Fetched 1474800 candles... (oldest: 2019-06-05 02:00:00)
  Fetched 1475000 candles... (oldest: 2019-06-04 18:20:00)
  Fetched 1475200 candles... (oldest: 2019-06-04 10:40:00)
  Fetched 1475400 candles... (oldest: 2019-06-04 03:00:00)
  Fetched 1475600 candles... (oldest: 2019-06-03 19:20:00)
  Fetched 1475800 candles... (oldest: 2019-06-03 11:40:00)
  Fetched 1476000 candles... (oldest: 2019-06-03 04:00:00)
  Fetched 1476200 candles... (oldest: 2019-06-02 20:20:0

  Fetched 1501000 candles... (oldest: 2019-04-24 04:05:00)
  Fetched 1501200 candles... (oldest: 2019-04-23 20:25:00)
  Fetched 1501400 candles... (oldest: 2019-04-23 12:45:00)
  Fetched 1501600 candles... (oldest: 2019-04-23 05:05:00)
  Fetched 1501800 candles... (oldest: 2019-04-22 21:25:00)
  Fetched 1502000 candles... (oldest: 2019-04-22 13:45:00)
  Fetched 1502200 candles... (oldest: 2019-04-22 06:05:00)
  Fetched 1502400 candles... (oldest: 2019-04-21 22:25:00)
  Fetched 1502600 candles... (oldest: 2019-04-21 14:45:00)
  Fetched 1502800 candles... (oldest: 2019-04-21 07:05:00)
  Fetched 1503000 candles... (oldest: 2019-04-20 23:25:00)
  Fetched 1503200 candles... (oldest: 2019-04-20 15:45:00)
  Fetched 1503400 candles... (oldest: 2019-04-20 08:05:00)
  Fetched 1503600 candles... (oldest: 2019-04-20 00:25:00)
  Fetched 1503800 candles... (oldest: 2019-04-19 16:45:00)
  Fetched 1504000 candles... (oldest: 2019-04-19 09:05:00)
  Fetched 1504200 candles... (oldest: 2019-04-19 01:25:0

  Fetched 1528800 candles... (oldest: 2019-03-09 18:00:00)
  Fetched 1529000 candles... (oldest: 2019-03-09 10:20:00)
  Fetched 1529200 candles... (oldest: 2019-03-09 02:40:00)
  Fetched 1529400 candles... (oldest: 2019-03-08 19:00:00)
  Fetched 1529600 candles... (oldest: 2019-03-08 11:20:00)
  Fetched 1529800 candles... (oldest: 2019-03-08 03:35:00)
  Fetched 1530000 candles... (oldest: 2019-03-07 19:50:00)
  Fetched 1530200 candles... (oldest: 2019-03-07 12:10:00)
  Fetched 1530400 candles... (oldest: 2019-03-07 04:15:00)
  Fetched 1530600 candles... (oldest: 2019-03-06 20:20:00)
  Fetched 1530800 candles... (oldest: 2019-03-06 12:25:00)
  Fetched 1531000 candles... (oldest: 2019-03-06 04:45:00)
  Fetched 1531200 candles... (oldest: 2019-03-05 21:05:00)
  Fetched 1531400 candles... (oldest: 2019-03-05 13:25:00)
  Fetched 1531600 candles... (oldest: 2019-03-05 05:45:00)
  Fetched 1531800 candles... (oldest: 2019-03-04 22:00:00)
  Fetched 1532000 candles... (oldest: 2019-03-04 14:15:0

  Fetched 1556800 candles... (oldest: 2019-01-23 10:20:00)
  Fetched 1557000 candles... (oldest: 2019-01-23 02:40:00)
  Fetched 1557200 candles... (oldest: 2019-01-22 19:00:00)
  Fetched 1557400 candles... (oldest: 2019-01-22 11:20:00)
  Fetched 1557600 candles... (oldest: 2019-01-22 03:30:00)
  Fetched 1557800 candles... (oldest: 2019-01-21 19:30:00)
  Fetched 1558000 candles... (oldest: 2019-01-21 11:35:00)
  Fetched 1558200 candles... (oldest: 2019-01-21 03:55:00)
  Fetched 1558400 candles... (oldest: 2019-01-20 20:15:00)
  Fetched 1558600 candles... (oldest: 2019-01-20 12:35:00)
  Fetched 1558800 candles... (oldest: 2019-01-20 04:55:00)
  Fetched 1559000 candles... (oldest: 2019-01-19 21:15:00)
  Fetched 1559200 candles... (oldest: 2019-01-19 13:35:00)
  Fetched 1559400 candles... (oldest: 2019-01-19 05:55:00)
  Fetched 1559600 candles... (oldest: 2019-01-18 22:10:00)
  Fetched 1559800 candles... (oldest: 2019-01-18 14:25:00)
  Fetched 1560000 candles... (oldest: 2019-01-18 06:45:0

In [1]:
import os
import time
import warnings
from datetime import datetime, timedelta
import pandas as pd
import requests
import yfinance as yf
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from binance.client import Client

warnings.filterwarnings('ignore')

# ==============================================================================
# [설정] 1시간봉 / 2020년 시작 / 저장 폴더 변경
# ==============================================================================
START_DATE = "2020-01-01"
OUTPUT_DIR = "./macro_data_1h"  # 폴더명 변경
os.makedirs(OUTPUT_DIR, exist_ok=True)

# [원칙] 실행 시점보다 미래인 데이터는 무조건 삭제
NOW = pd.Timestamp.now()

UPBIT_TICKERS = {
    'KRW-BTC': ('BTC', 'BTC'), 'KRW-ETH': ('ETH', 'ETH'), 'KRW-XRP': ('XRP', 'XRP'),
    'KRW-SOL': ('SOL', 'SOL'), 'KRW-ADA': ('ADA', 'ADA'), 'KRW-DOGE': ('DOGE', 'DOGE'),
    'KRW-AVAX': ('AVAX', 'AVAX'), 'KRW-DOT': ('DOT', 'DOT')
}

BINANCE_SYMBOLS = {
    'BTCUSDT': 'BTC', 'ETHUSDT': 'ETH', 'XRPUSDT': 'XRP', 'SOLUSDT': 'SOL',
    'ADAUSDT': 'ADA', 'DOGEUSDT': 'DOGE', 'AVAXUSDT': 'AVAX', 'DOTUSDT': 'DOT'
}

MACRO_TICKERS = {
    'DX-Y.NYB': 'DXY', 'GC=F': 'GOLD', '^VIX': 'VIX', '^GSPC': 'SP500'
}

DEFI_PROTOCOLS = ['makerdao', 'lido', 'aave', 'uniswap', 'curve-dex']
L2_CHAINS = ['Arbitrum', 'Optimism', 'Base', 'zkSync Era']

def get_session():
    session = requests.Session()
    session.headers.update({
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json"
    })
    retry = Retry(total=5, backoff_factor=2, status_forcelist=[429, 500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retry))
    return session

# -----------------------------------------------------------------------------
# [1] Upbit (기준 데이터) - 1시간봉으로 변경
# -----------------------------------------------------------------------------
def collect_upbit_crypto_prices_1h():
    print(f"\n[1/7] Collecting Upbit (1H, Safe Cut: <= NOW)...")
    session = get_session()
    start_dt = pd.to_datetime(START_DATE)
    merged_df = None
    
    for market, (symbol, _) in UPBIT_TICKERS.items():
        try:
            all_candles = []
            to_date = None
            while True:
                # [수정] 240 -> 60 (1시간)
                url = "https://api.upbit.com/v1/candles/minutes/60"
                params = {'market': market, 'count': 200}
                if to_date: params['to'] = to_date
                resp = session.get(url, params=params, timeout=10)
                candles = resp.json()
                if not candles: break
                all_candles.extend(candles)
                if pd.to_datetime(candles[-1]['candle_date_time_kst']) <= start_dt: break
                to_date = candles[-1]['candle_date_time_utc']
                time.sleep(0.1)
            
            if not all_candles: continue
            
            df = pd.DataFrame(all_candles)
            df['timestamp'] = pd.to_datetime(df['candle_date_time_kst'])
            
            # [원칙 적용] 미래 데이터 삭제
            df = df[df['timestamp'] <= NOW]
            
            df = df.rename(columns={
                'opening_price': f'{symbol}_Open', 'high_price': f'{symbol}_High',
                'low_price': f'{symbol}_Low', 'trade_price': f'{symbol}_Close',
                'candle_acc_trade_volume': f'{symbol}_Volume'
            })
            df = df[['timestamp', f'{symbol}_Open', f'{symbol}_High', f'{symbol}_Low', f'{symbol}_Close', f'{symbol}_Volume']]
            df = df.sort_values('timestamp').drop_duplicates('timestamp')
            df = df[df['timestamp'] >= start_dt]
            
            merged_df = df if merged_df is None else pd.merge(merged_df, df, on='timestamp', how='outer')
            print(f"  - {symbol}: {len(df)} rows")
        except Exception as e: print(f"  - {symbol}: Failed {e}")
    
    if merged_df is not None:
        merged_df = merged_df.sort_values('timestamp')
        # [수정] 파일명 1h로 변경
        merged_df.to_csv(os.path.join(OUTPUT_DIR, "crypto_1h_kst.csv"), index=False)

# -----------------------------------------------------------------------------
# [2] Binance (KST Aligned & Safe Cut) - 1시간봉으로 변경
# -----------------------------------------------------------------------------
def collect_binance_crypto_prices_1h():
    print(f"\n[2/7] Collecting Binance (1H, KST Aligned & Safe Cut)...")
    client = Client("", "")
    start_ms = int(datetime.strptime(START_DATE, "%Y-%m-%d").timestamp() * 1000)
    merged_df = None

    for symbol, base in BINANCE_SYMBOLS.items():
        try:
            # [수정] 4HOUR -> 1HOUR
            klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR, start_ms)
            df = pd.DataFrame(klines, columns=["open_time", "o", "h", "l", "c", "v", "ct", "qav", "nt", "tbb", "tbq", "ig"])
            
            # UTC -> KST (+9h)
            df["timestamp"] = pd.to_datetime(df["open_time"], unit="ms") + pd.Timedelta(hours=9)
            
            # [원칙 적용] 미래 데이터 삭제
            df = df[df['timestamp'] <= NOW]
            
            # [삭제] 4시간 정렬 코드 제거 (모든 시간 사용)
            # df = df[df['timestamp'].dt.hour % 4 == 1] 
            
            df = df.rename(columns={
                "o": f"{base}_Bin_Open", "h": f"{base}_Bin_High",
                "l": f"{base}_Bin_Low", "c": f"{base}_Bin_Close",
                "v": f"{base}_Bin_Volume"
            })
            df = df[['timestamp', f'{base}_Bin_Open', f'{base}_Bin_High', f'{base}_Bin_Low', f'{base}_Bin_Close', f'{base}_Bin_Vol']]
            df = df[df["timestamp"] >= pd.to_datetime(START_DATE)]
            
            merged_df = df if merged_df is None else pd.merge(merged_df, df, on="timestamp", how="outer")
            print(f"  - {symbol}: {len(df)} rows")
        except: pass
            
    if merged_df is not None:
        merged_df = merged_df.sort_values("timestamp")
        # [수정] 파일명 1h로 변경
        merged_df.to_csv(os.path.join(OUTPUT_DIR, "crypto_binance_1h_kst.csv"), index=False)

# -----------------------------------------------------------------------------
# [3] Macro (Shifted & Safe Cut) - 1시간봉 전체 사용
# -----------------------------------------------------------------------------
def collect_macro_indicators_1h():
    print(f"\n[3/7] Collecting Macro (1H, Shifted & Safe Cut)...")
    for ticker, name in MACRO_TICKERS.items():
        try:
            df = yf.download(ticker, start=START_DATE, end=None, progress=False, interval='1d')
            if isinstance(df.columns, pd.MultiIndex): df = df.xs('Close', level=0, axis=1)
            else: df = df[['Close']]
            
            df.index = pd.to_datetime(df.index).tz_localize(None)
            df.columns = [name]
            df.index.name = 'timestamp'

            # 1일 + 9시간 Shift
            df.index = df.index + pd.Timedelta(days=1, hours=9)
            
            # [원칙 적용]
            df = df[df.index <= NOW]
            
            # ffill로 1시간 단위 채움
            full_idx = pd.date_range(start=df.index[0], end=NOW, freq='1H')
            df = df.reindex(full_idx, method='ffill')
            
            # [삭제] 4시간 정렬 코드 제거
            # df_4h = df[df.index.hour % 4 == 1].copy()
            
            df_1h = df[df.index >= pd.to_datetime(START_DATE)]
            df_1h.to_csv(os.path.join(OUTPUT_DIR, f"{name}_1h.csv"))
            print(f"  - {name}: {len(df_1h)} rows")
        except Exception as e:
            print(f"  - {name}: Failed {e}")

# -----------------------------------------------------------------------------
# [4] Fear & Greed (Shifted & Safe Cut)
# -----------------------------------------------------------------------------
def collect_fear_greed_1h():
    print(f"\n[4/7] Collecting Fear & Greed (1H, Shifted & Safe Cut)...")
    try:
        session = get_session()
        resp = session.get("https://api.alternative.me/fng/?limit=4000&format=json", timeout=10)
        data = resp.json()['data']
        
        df = pd.DataFrame(data)
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
        df = df[['timestamp', 'value']].rename(columns={'value': 'fear_greed'})
        df['fear_greed'] = df['fear_greed'].astype(float)
        df = df.set_index('timestamp').sort_index()
        
        df.index = df.index + pd.Timedelta(days=1, hours=9)
        df = df[df.index <= NOW]
        
        full_idx = pd.date_range(start=df.index[0], end=NOW, freq='1H')
        df = df.reindex(full_idx, method='ffill')
        
        # [삭제] 4시간 정렬 제거
        df_1h = df[df.index >= pd.to_datetime(START_DATE)].reset_index()
        
        df_1h.to_csv(os.path.join(OUTPUT_DIR, "fear_greed_1h.csv"), index=False)
        print(f"  - Fear & Greed: {len(df_1h)} rows")
    except Exception as e:
        print(f"  - Fear & Greed: Failed {e}")

# -----------------------------------------------------------------------------
# [5] Funding Rate (KST Aligned & Safe Cut)
# -----------------------------------------------------------------------------
def collect_funding_rate_1h():
    print(f"\n[5/7] Collecting ETH Funding Rate (1H, KST Aligned & Safe Cut)...")
    try:
        client = Client("", "")
        funding_rates = []
        start_ts = int(datetime.strptime(START_DATE, "%Y-%m-%d").timestamp() * 1000)
        end_ts = int(datetime.now().timestamp() * 1000)
        
        while start_ts < end_ts:
            rates = client.futures_funding_rate(symbol='ETHUSDT', startTime=start_ts, limit=1000)
            if not rates: break
            funding_rates.extend(rates)
            start_ts = rates[-1]['fundingTime'] + 1
            time.sleep(0.1)
        
        df = pd.DataFrame(funding_rates)
        df['timestamp'] = pd.to_datetime(df['fundingTime'], unit='ms') + pd.Timedelta(hours=9)
        df['fundingRate'] = df['fundingRate'].astype(float)
        df = df[['timestamp', 'fundingRate']].sort_values('timestamp').set_index('timestamp')
        
        df = df[df.index <= NOW]
        
        full_idx = pd.date_range(start=df.index[0], end=NOW, freq='1H')
        df = df.reindex(full_idx, method='ffill')
        
        # [삭제] 4시간 정렬 제거
        df_1h = df[df.index >= pd.to_datetime(START_DATE)].reset_index()
        
        df_1h.to_csv(os.path.join(OUTPUT_DIR, "eth_funding_rate_1h.csv"), index=False)
        print(f"  - Funding Rate: {len(df_1h)} rows")
    except Exception as e:
        print(f"  - Funding Rate: Failed {e}")

# -----------------------------------------------------------------------------
# [6, 7] TVL (Shifted & Safe Cut)
# -----------------------------------------------------------------------------
def collect_tvl_all_1h():
    print(f"\n[6~7/7] Collecting TVL (1H, Shifted & Safe Cut)...")
    
    def get_data_manual(url):
        session = get_session()
        for i in range(3):
            try:
                time.sleep(2) 
                resp = session.get(url, timeout=30)
                if resp.status_code == 200: return resp.json()
                time.sleep(5)
            except: time.sleep(5)
        return None

    def process_and_save(df, filename, col_name):
        if df is None or df.empty: return
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        if df['timestamp'].dt.tz is not None:
            df['timestamp'] = df['timestamp'].dt.tz_localize(None)
            
        df['timestamp'] = df['timestamp'] + pd.Timedelta(days=1, hours=9)
        df = df.sort_values('timestamp').drop_duplicates(subset=['timestamp']).set_index('timestamp')
        df = df[df.index <= NOW]
        
        full_idx = pd.date_range(start=df.index[0], end=NOW, freq='1H')
        df = df.reindex(full_idx, method='ffill')
        
        # [삭제] 4시간 정렬 제거
        df_1h = df[df.index >= pd.to_datetime(START_DATE)].reset_index()
        
        # [수정] 파일명 1h로 저장
        # filename 자체를 호출할 때 '_1h.csv'로 넘겨주거나 여기서 치환
        new_filename = filename.replace('_4h.csv', '_1h.csv')
        df_1h.to_csv(os.path.join(OUTPUT_DIR, new_filename), index=False)
        print(f"  - {col_name}: {len(df_1h)} rows")

    # ETH Chain
    try:
        data = get_data_manual("https://api.llama.fi/v2/historicalChainTvl/Ethereum")
        if data:
            df = pd.DataFrame(data)
            df['timestamp'] = pd.to_datetime(df['date'], unit='s')
            df = df.rename(columns={'tvl': 'eth_chain_tvl'})[['timestamp', 'eth_chain_tvl']]
            process_and_save(df, 'eth_chain_tvl_4h.csv', 'ETH Chain TVL')
    except: pass

    # Protocols
    for protocol in DEFI_PROTOCOLS:
        try:
            data = get_data_manual(f"https://api.llama.fi/protocol/{protocol}")
            if data:
                chain_data = data.get('chainTvls', {}).get('Ethereum', {}).get('tvl', [])
                if not chain_data: chain_data = data.get('tvl', [])
                if chain_data:
                    df = pd.DataFrame(chain_data)
                    df['timestamp'] = pd.to_datetime(df['date'], unit='s')
                    val_col = 'totalLiquidityUSD' if 'totalLiquidityUSD' in df.columns else 'tvl'
                    df = df.rename(columns={val_col: f'{protocol}_eth_tvl'})
                    process_and_save(df[['timestamp', f'{protocol}_eth_tvl']], f'{protocol}_eth_tvl_4h.csv', protocol)
        except: pass

    # L2 Chains
    for chain in L2_CHAINS:
        try:
            chain_name = chain.replace(' ', '_').lower()
            data = get_data_manual(f"https://api.llama.fi/v2/historicalChainTvl/{chain}")
            if data:
                df = pd.DataFrame(data)
                df['timestamp'] = pd.to_datetime(df['date'], unit='s')
                df = df.rename(columns={'tvl': f'{chain_name}_tvl'})
                process_and_save(df[['timestamp', f'{chain_name}_tvl']], f'{chain_name}_tvl_4h.csv', chain)
        except: pass
    
    # USDT Mcap
    try:
        data = get_data_manual("https://stablecoins.llama.fi/stablecoincharts/Ethereum?stablecoin=1")
        if data:
            df = pd.DataFrame(data)
            if 'totalCirculatingUSD' in df.columns:
                df['usdt_eth_mcap'] = df['totalCirculatingUSD'].apply(
                    lambda x: x.get('peggedUSD') if isinstance(x, dict) else x
                )
                df['timestamp'] = pd.to_datetime(df['date'], unit='s')
                process_and_save(df[['timestamp', 'usdt_eth_mcap']], 'usdt_eth_mcap_4h.csv', 'USDT Mcap')
    except Exception as e:
        print(f"  - USDT Mcap: Failed ({str(e)[:50]})")

# -----------------------------------------------------------------------------
# Run All (1시간봉 수집)
# -----------------------------------------------------------------------------
print("🚀 Starting Final Pipeline (1H, 2020-01-01, Safe Cut)...")
collect_upbit_crypto_prices_1h()
collect_binance_crypto_prices_1h()
collect_macro_indicators_1h()
collect_fear_greed_1h()
collect_funding_rate_1h()
collect_tvl_all_1h()
print(f"✅ All Done. Files saved in {OUTPUT_DIR}")


🚀 Starting Final Pipeline (1H, 2020-01-01, Safe Cut)...

[1/7] Collecting Upbit (1H, Safe Cut: <= NOW)...
  - BTC: 51919 rows
  - ETH: 51916 rows
  - XRP: 51914 rows
  - SOL: 36251 rows
  - ADA: 51912 rows
  - DOGE: 41828 rows
  - AVAX: 33457 rows
  - DOT: 45219 rows

[2/7] Collecting Binance (1H, KST Aligned & Safe Cut)...

[3/7] Collecting Macro (1H, Shifted & Safe Cut)...
  - DXY: 51942 rows
  - GOLD: 51942 rows
  - VIX: 51942 rows
  - SP500: 51942 rows

[4/7] Collecting Fear & Greed (1H, Shifted & Safe Cut)...
  - Fear & Greed: 51999 rows

[5/7] Collecting ETH Funding Rate (1H, KST Aligned & Safe Cut)...
  - Funding Rate: 51998 rows

[6~7/7] Collecting TVL (1H, Shifted & Safe Cut)...
  - ETH Chain TVL: 51999 rows
  - makerdao: 51999 rows
  - lido: 43471 rows
  - aave: 48606 rows
  - uniswap: 51999 rows
  - curve-dex: 51030 rows
  - Arbitrum: 39486 rows
  - Optimism: 38526 rows
  - Base: 21702 rows
  - zkSync Era: 31278 rows
  - USDT Mcap: 51999 rows
✅ All Done. Files saved in ./mac